**This notebook reproduces the training of the classical methods. The 'USE_FS_Columns' parameter inside the cells indicate whether the feature selection approach is used in the model.**

In [ ]:
import os 
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
main_path = os.path.abspath(os.path.dirname(os.getcwd()))
sys.path.append(main_path)
csvs_dir_ensemble = os.path.abspath(os.path.join('..','data','csvs_for_classic_methods'))

from models.classic_models import *

fs_columns = ['GFS_ysutr', 'ARPEGE_ysumc', 'GEM_uwtc', 'GEM_myjtr', 'GFS_uwmc', 
              'GFS_uwtr', 'ARPEGE_uwmc', 'ARPEGE_uwtc', 'GFS_uwtc', 'GEM_mynn2mc']

# Climatological CSGD calculation

In [ ]:
import os 
list_station_names = set(name[:-len(name.split('_')[-1]) -1] for name in os.listdir('../data/csvs_for_classic_methods') if name.endswith('.csv'))
len(list_station_names)

In [ ]:
USE_FS_Columns = True #It doesn't matter in climatological CSGD
params_0_list = [(l,m,n,p) for l in [0.005,0.01,0.015,0.02] for m in [0.01,0.02,0.05,0.1] for n in np.arange(1,2,0.25) for p in [0.01,0.05,0.1,0.15]]

total_n = 0
total_crps = 0
total_mae = 0
total_bs = 0
for station_name in list_station_names:
    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')
    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file).dropna()
    ground_truth_train = ensemb_train_df.iloc[:,1].values
    ground_truth_test = ensemb_test_df.iloc[:,1].values
    climat_csgd = CSGD(verbose=False)
    params0 = climat_csgd.calc_initial_values(ground_truth = ground_truth_train, method='paper')
    climat_csgd.fit_climatological(ground_truth = ground_truth_train, init_params = params0)#,params_0 = [0.01,0.02,1.5,0.1])
    crps, mae, mse, bs = climat_csgd.predict(ground_truth_test, metric = 'all')
    total_n += len(ground_truth_test)
    total_crps += crps*len(ground_truth_test)
    total_mae += mae*len(ground_truth_test)
    total_bs += bs*len(ground_truth_test)
    print(f'{station_name} MAE: {mae}')

crps_value = total_crps/total_n
mae_value = total_mae/total_n
bs_value = total_bs/total_n
print(total_n)
print(f'Climatological. CRPS:{crps_value} MAE:{mae_value} BS:{bs_value}')

# Predictive CSGD calculation

Using SLSQP with constraints:

In [ ]:
USE_FS_Columns = True 

params_0_list = [(l,m,n,p) for l in [0.005,0.01,0.015,0.02] for m in [0.01,0.02,0.05,0.1] for n in np.arange(1,2,0.25) for p in [0.01,0.05,0.1,0.15]]
total_crps = 0
total_mae = 0
total_bs = 0
total_n = 0
estaciones_pesos = {}
for i,station_name in enumerate(list_station_names):
    print(f'·{i + 1}/{len(list_station_names)} {station_name}:')

    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_val_file = os.path.join(csvs_dir_ensemble ,station_name + '_val.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')

    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_val_df = pd.read_csv(ensemb_val_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file).dropna()

    ground_truth_train = ensemb_train_df.iloc[:,1].values
    ground_truth_val = ensemb_val_df.iloc[:,1].values
    ground_truth_test = ensemb_test_df.iloc[:,1].values

    if USE_FS_Columns:
        wrf_ensemble_train = ensemb_train_df[fs_columns].values
        wrf_ensemble_val = ensemb_val_df[fs_columns].values
        wrf_ensemble_test = ensemb_test_df[fs_columns].values
    else:
        wrf_ensemble_train = ensemb_train_df.iloc[:,2:].values
        wrf_ensemble_val = ensemb_val_df.iloc[:,2:].values
        wrf_ensemble_test = ensemb_test_df.iloc[:,2:].values   
    
    n_station = len(ground_truth_test)
    best_crps_val = 1
    for params_0 in params_0_list:
        pred_csgd = PredictiveCSGD(verbose=False)
        pred_csgd.fit(ground_truth_train,wrf_ensemble_train, params_0 = params_0)#,params_0 = [0.01,0.02,1.5,0.1])
        crps_val = pred_csgd.predict(ground_truth_val,wrf_ensemble_val)
        if crps_val < best_crps_val:
            best_crps_val = crps_val
            best_pred_csgd = pred_csgd
            best_init_params = params_0

    crps_test, mae_test, mse_test, bs_test = best_pred_csgd.predict(ground_truth_test,wrf_ensemble_test, metrics = 'all')
    print(f' Best params_init: {best_init_params}, best_params_fitted: {best_pred_csgd.fitted_params} \n crps: {crps_test} mae: {mae_test} bs: {bs_test}')
    total_n += n_station
    total_crps += crps_test * n_station
    total_mae += mae_test * n_station
    total_bs += bs_test * n_station
    estaciones_pesos[station_name] = {'pred':best_pred_csgd.fitted_params,'climat':best_pred_csgd.fitted_climat_params}

crps_final = total_crps/total_n
mae_final = total_mae / total_n
bs_final = total_bs / total_n
print(f' crps_final = {crps_final} mae = {mae_final} bs = {bs_final}')

"""import pickle
# Guardamos el diccionario en un archivo .pkl usando pickle
with open('PredCSGD_pesos.pkl', 'wb') as f:
    pickle.dump(estaciones_pesos, f)"""

## Analog Ensemble

In [ ]:
USE_FS_Columns = True
total_n = 0
total_crps = 0
total_crps_ensemb = 0
total_mae = 0
total_mse = 0
total_bs = 0
for idx,station_name in enumerate(list_station_names):
    print(f'·{idx}/{len(list_station_names)}.{station_name}:')
    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_val_file = os.path.join(csvs_dir_ensemble ,station_name + '_val.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')
    
    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_val_df = pd.read_csv(ensemb_val_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file).dropna()
    ground_truth_train = ensemb_train_df.iloc[:,1].values
    ground_truth_test = ensemb_test_df.iloc[:,1].values

    if USE_FS_Columns:
        X_ensemb_train_df = ensemb_train_df.loc[:,fs_columns]
        X_ensemb_val_df = ensemb_val_df.loc[:,fs_columns]
        X_ensemb_test_df = ensemb_test_df.loc[:,fs_columns]
    else:
        X_ensemb_train_df = ensemb_train_df.iloc[:,2:]
        X_ensemb_val_df = ensemb_val_df.iloc[:,2:]
        X_ensemb_test_df = ensemb_test_df.iloc[:,2:]

    AnEn = AnalogEnsemble(t_window=3)
    AnEn.fit(X_train = X_ensemb_train_df, y_train = ensemb_train_df.iloc[:,1], X_val = X_ensemb_val_df, y_val = ensemb_val_df.iloc[:,1],
              n_members_options=[25,35,50], masked_columns = fs_columns if USE_FS_Columns else None) 
    crps, mae, mse, bs = AnEn.predict(X_test = ensemb_test_df.iloc[:,2:], y_test = ensemb_test_df.iloc[:,1])
    crps_ensemb = np.mean(crps_ensemble(ground_truth_test, X_ensemb_test_df.values))
    total_n += len(ground_truth_test)
    total_crps += crps*len(ground_truth_test)
    total_crps_ensemb += crps_ensemb*len(ground_truth_test)
    total_mae += mae*len(ground_truth_test)
    total_mse += mse*len(ground_truth_test)
    total_bs += bs*len(ground_truth_test)
    print(f' CRPS: {crps} CRPS_ensemb: {crps_ensemb}')
    print(f' BS {bs} MAE {mae} MSE {mse}')
    print('')
crps_value = total_crps/total_n
mae_value = total_mae/total_n
mse_value = total_mse/total_n
bs_value = total_bs/total_n
print(total_crps_ensemb/total_n)
print(total_n)
print(f'Climatological. CRPS:{crps_value} MAE:{mae_value} MSE:{mse_value} BS:{bs_value}')

## ANN-CSGD

In [ ]:
USE_FS_Columns = True
total_crps = 0
total_mae = 0
total_bs = 0
total_n = 0
estaciones_pesos = {}
for i,station_name in enumerate(list_station_names):
    print(f'·{i + 1}/{len(list_station_names)} {station_name}:')

    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_val_file = os.path.join(csvs_dir_ensemble ,station_name + '_val.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')

    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_val_df = pd.read_csv(ensemb_val_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file).dropna()

    ground_truth_train = ensemb_train_df.iloc[:,1].values
    ground_truth_val = ensemb_val_df.iloc[:,1].values
    ground_truth_test = ensemb_test_df.iloc[:,1].values
    if USE_FS_Columns:
        wrf_ensemble_train = ensemb_train_df[fs_columns].values
        wrf_ensemble_val = ensemb_val_df[fs_columns].values
        wrf_ensemble_test = ensemb_test_df[fs_columns].values
    else:
        wrf_ensemble_train = ensemb_train_df.iloc[:,2:].values
        wrf_ensemble_val = ensemb_val_df.iloc[:,2:].values
        wrf_ensemble_test = ensemb_test_df.iloc[:,2:].values
    n_station = len(ground_truth_test)
    best_crps_val = 1
    anncsgd = ANNCSGD(verbose=False,learning_rate=1e-3, input_dim = wrf_ensemble_train.shape[1])
    anncsgd.fit(wrf_ensemble_train,ground_truth_train,wrf_ensemble_val,ground_truth_val, seed = 11) 
    crps_test,mae_test, bs_test, var_bs_test = anncsgd.predict(wrf_ensemble_test,ground_truth_test)
    crps_ensemb = np.mean(crps_ensemble(ground_truth_test, ensemb_test_df.iloc[:,2:].values))

    print(f' crps: {crps_test} mae: {mae_test} bs: {bs_test} +- {np.sqrt(var_bs_test)}, crps_ensemble: {crps_ensemb}')
    total_n += n_station
    total_crps += crps_test * n_station
    total_mae += mae_test * n_station
    total_bs += bs_test * n_station
    estaciones_pesos[station_name] = deepcopy(anncsgd.state_dict())

torch.save(estaciones_pesos,'weights_10FS_anncsgd.pt')
crps_final = total_crps/total_n
mae_final = total_mae / total_n
bs_final = total_bs / total_n
print(f' crps_final = {crps_final} mae = {mae_final} bs = {bs_final}')